#How To Build a Model: Step By Step


In [1]:
#Pytorch Imports
%load_ext autoreload
%autoreload 2

import torch
from torch import nn

from torch.utils.data import DataLoader
from torchtext.data import get_tokenizer
from torchtext.datasets import IMDB
from torchtext.transforms import ToTensor
from torchtext.vocab import build_vocab_from_iterator
#Pandas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split
import time

#MOdel Architectures
from NNDL.transformer.architecture import TransformerModel,TransformerMetaModel
from NNDL.RNN.architecture import RNN,MyNetwork
from NNDL.TestModel.structure import FCNN
import NNDL.Utils.solver as solver
from NNDL.Utils.weight_tracker import ActivationMonitor



c:\Users\haoxu\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from transformers import AutoTokenizer

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

Pick Your Universal Hyperparamaters:

In [3]:
batch_size = 32
epochs = 7
input_dim = 100
hidden_dim = 50
output_dim = 10
n_filters=30
filter_size=5
drop_frac=0.5
embed_dim = 50
train_size = 0.2
test_size = 0.1
meta_train_size = 0.2
meta_test_size = 0.1
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
torch.cuda.amp.autocast()

print(f"==>> torch.cuda.is_available(): {torch.cuda.is_available()}")
print(device)
device = "cpu"

==>> torch.cuda.is_available(): True
cuda


# Setup Your Model Workspace:

Your model architecture, and all of it's relevent code, will go in a folder at the location ./NNDL/(Your Model Name Here)


# Setup Your Dataset/Data Loaders Here

Pytorch usually takes a csv file. You need to write a function( For naming convention let's call it *create_torch_datasets()* ) that takes the universal dataset kyle got from parsing and seperates/loads it into data that you want your model to train on




In [4]:
#Create Custom Dataset Class:
from torchtext.data import get_tokenizer
from torchtext.vocab import GloVe
from transformers import BertTokenizer
from PIL import Image
import requests
from torchvision.io import read_image
import torchvision.transforms as transforms

# Load the pre-trained BERT tokenizer
tokenizer = get_tokenizer("basic_english")

# Input text
text = "Hello, how are you doing?"

vec = GloVe(name='6B', dim=embed_dim)
class FullDataset(Dataset):
    def __init__(self, data, root_dir):
        self.root_dir = root_dir
        self.data = data
        self.mapping = {'arts, crafts & sewing': 0,'books': 1, 'clothing, shoes & jewelry': 2,'electronics': 3, 'grocery & gourmet food': 4,'health & personal care': 5, 'musical instruments': 6, 'patio, lawn & garden': 7,'sports & outdoors': 8, 'toys & games': 9}
        self.cur_map = 0
        self.print_data=False
    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
        if  type(self.data.iloc[index, 0]) == float:
            t="N/A"     
            print('N/A Example  In dataset')
        else:
            t=self.data.iloc[index, 0]
        tokens = tokenizer.tokenize(t)
        token_length=input_dim
        tokens=tokens+[""] * (token_length-len(tokens))  if len(tokens)<token_length else tokens[:token_length]
        tokens_emb = [vec.stoi.get(token, 0) for token in tokens]
        img_link = os.path.join(self.data.iloc[index, 1])
        category_text = os.path.join(self.data.iloc[index, 2])
        if category_text in self.mapping:
            category = self.mapping[category_text]
        else:
            print(category_text)
            self.mapping[category_text] = self.cur_map
            print("HOUSTON WE GOT A PROBLEM")
            print(self.mapping)
            category = self.cur_map
            self.cur_map+=1
        try:
            image = Image.open(requests.get(img_link, stream=True).raw) 
        except:
            print(img_link)
        image= image.resize((64,64))
        transform = transforms.Compose([transforms.PILToTensor()])
        torch_image = transform(image)
        if torch_image.shape[0] != 3:
            print(img_link)
            print("WTF")
            print(torch_image.shape)
            x=torch.zeros_like(torch_image)
            torch_image=torch.cat([torch_image,x,x],dim=0)
            print(f"Post correction shape:{torch_image.shape}")
        if self.print_data==True:
            print(f"img_link:{img_link}")
            print(f"Name+Desciption{t}")
        return torch.tensor(tokens_emb),torch_image,category


In [5]:
from transformers import BertTokenizer

# Load the pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Input text
text = "Hello, how are you doing?"

# Tokenize the text
tokens = tokenizer.tokenize(text)
tokens_emb = [vec.stoi[token] for token in tokens]
print(torch.tensor(tokens_emb))
# Convert tokens to numerical IDs
token_ids = tokenizer.convert_tokens_to_ids(tokens)

# Print the results
print("Original Text:", text)
print("Tokens:", tokens)
print("Token IDs:", torch.tensor(token_ids))

tensor([13075,     1,   197,    32,    81,   914,   188])
Original Text: Hello, how are you doing?
Tokens: ['hello', ',', 'how', 'are', 'you', 'doing', '?']
Token IDs: tensor([7592, 1010, 2129, 2024, 2017, 2725, 1029])


In [6]:
#For Transformer,CNN,RNN
d = pd.read_csv(r'data/test_remove_filtered.csv')

full_dataset = FullDataset(data=d,root_dir='/')

train,test,meta = torch.utils.data.random_split(full_dataset,[train_size,test_size,1-(train_size+test_size)])
train_dataloader =  DataLoader(train, batch_size=batch_size,
                        shuffle=True, num_workers=0, drop_last=True)
test_dataloader =  DataLoader(test, batch_size=batch_size,
                        shuffle=True, num_workers=0, drop_last=True)
meta_train,meta_test,trash = torch.utils.data.random_split(meta,[meta_train_size,meta_test_size,1-(meta_train_size+meta_test_size)])
meta_train_dataloader = DataLoader(meta_train, batch_size=batch_size,
                        shuffle=True, num_workers=0, drop_last=True)
meta_test_dataloader = DataLoader(meta_test, batch_size=batch_size,
                        shuffle=True, num_workers=0, drop_last=True)

#vocab= build_vocab_from_iterator(train_dataloader,specials=["<unk>"]).to(device)
#vocab.set_default_index(vocab["<unk>"])
# train_iter= IMDB(split="train")
# test_iter = IMDB(split="test")
# tokenizer = get_tokenizer("basic_english")
# def yield_tokens(data_iter):
#     for _, text in data_iter:
#         yield tokenizer(text)
# train_tokens = []
# vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
# vocab.set_default_index(vocab["<unk>"])
# test_tokens = []
# test_tokens = build_vocab_from_iterator(yield_tokens(test_iter), specials=["<unk>"])



Put the dataset into data loader, and check the shape, make sure it's how you want it.


# Setup Model Architecture

1. Create your model architecture in your folder
2. Pick your loss function and optimizer

In [7]:

model1 = TransformerModel(input_dim,embed_dim,output_dim).to(device)
model2 = MyNetwork(embed_dim,n_filters,filter_size,drop_frac,output_dim,embed_dim).to(device)

loss_fn = nn.CrossEntropyLoss()



# Test the model Ensembles

In [8]:
import functools
import tensorflow as tf

top3_acc = functools.partial(tf.keras.metrics.top_k_categorical_accuracy, k=3)
top3_acc.__name__ = 'top3_acc'
top2_acc = functools.partial(tf.keras.metrics.top_k_categorical_accuracy, k=2)
top2_acc.__name__ = 'top2_acc'
custom_objects = {'top3_acc':top3_acc, 'top2_acc':top2_acc}


In [9]:
state_dict = torch.load(f"models\TransformerModel/0.25-0.05_epochs-0.pt")
model1.load_state_dict(state_dict)
state_dict = torch.load(f"models\MyNetwork/0.25-0.05_epochs-0.pt")
model2.load_state_dict(state_dict)

#Use Tensorflow Model
t_model = tf.keras.models.load_model("models\image\gln_model_Qc0Yq7.keras",custom_objects=custom_objects)
t_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model_list= [model1,model2]
weights = [0.5,0.25,0.25]
solver.test_ensemble(test_dataloader, model_list, loss_fn,t_model,weights)

Beginnning inference
Testing Prediction
Progress: 1/147,accuracy:0.6,test_loss:0.939146
Testing Prediction
Progress: 2/147,accuracy:1.0,test_loss:0.889791
Testing Prediction
Progress: 3/147,accuracy:1.5,test_loss:0.877443
Testing Prediction
Progress: 4/147,accuracy:2.1,test_loss:0.861989
http://ecx.images-amazon.com/images/I/51d87SIQcHL._SY300_.jpg


UnboundLocalError: local variable 'image' referenced before assignment

In [ ]:

#solver.test(test_dataloader,model1,loss_fn)

In [ ]:
#meta_model = FCNN(output_dim*(len(model_list)+1),hidden_dim*(len(model_list)+1),output_dim).to(device)
#train a Meta MOdel
meta_model=TransformerMetaModel(len(model_list)+1,output_dim,output_dim)
optimizer = torch.optim.Adam(meta_model.parameters(),lr=1e-3)
for t in range(epochs):
    #with torch.autograd.detect_anomaly():
        print(f"Epoch {t+1}\n-------------------------------")
        solver.train_stack(meta_train_dataloader,model_list,loss_fn,optimizer,meta_model,t_model)
        torch.save(meta_model.state_dict(), f"models\{type(meta_model).__name__}\{meta_train_size}-{meta_test_size}_epochs-{t}.pt")
        torch.cuda.empty_cache()
        solver.test_stack(meta_train_dataloader,model_list,loss_fn,meta_model,t_model)
        


Epoch 1
-------------------------------
Training Meta Model


KeyboardInterrupt: 

In [ ]:
#Sanity Check With Single Examples
with torch.no_grad():
    class_names=['arts, crafts & sewing','books', 'clothing, shoes & jewelry','electronics', 'grocery & gourmet food','health & personal care', 'musical instruments', 'patio, lawn & garden','sports & outdoors','toys & games']
    X_text,X_img, actual_label = next(iter(meta_test_dataloader))
    actual_label = actual_label[0]
    #PRedicting USing Models
    X_img=torch.transpose(X_img,1,3)
    X_img = X_img.numpy()
    X_img = tf.convert_to_tensor(X_img)
    X_img=tf.cast(X_img,tf.float32)
    X_img=tf.keras.applications.inception_v3.preprocess_input(X_img)
    pred_by_model = []
    for model in model_list:
        out = model(X_text)
        pred_by_model.append(out)
    t_out=t_model.predict(X_img)
    t_out=torch.tensor(t_out)
    pred_by_model.append(t_out)
    for it,weight in enumerate(weights):
        pred_by_model[it]*=weight
    stacked_tensor = torch.stack(pred_by_model)
    pred = torch.sum(stacked_tensor,axis=0)
    pred=np.argmax(np.array(pred), axis=1)
    print(f"The Meta Model Predicted:{class_names[pred[0]]},while the actual class was {class_names[actual_label]}")

KeyboardInterrupt: 

In [ ]:
#Train the stacking meta model
meta_model= FCNN(len(model_list),len(model_list),1)
solver.train(meta_model)
solver.test(test_dataloader,meta_model,loss_fn)

Epoch 1
-------------------------------
loss: 2.465192  [  256/125000]
Using GPU: NVIDIA GeForce RTX 3070
GPU Memory Usage:
Allocated: 0.1 GB
Cached:    0.1 GB
loss: 0.949329  [33024/125000]
Using GPU: NVIDIA GeForce RTX 3070
GPU Memory Usage:
Allocated: 1.3 GB
Cached:    1.8 GB
loss: 0.912446  [65792/125000]
Using GPU: NVIDIA GeForce RTX 3070
GPU Memory Usage:
Allocated: 2.6 GB
Cached:    3.1 GB
loss: 0.763514  [98560/125000]
Using GPU: NVIDIA GeForce RTX 3070
GPU Memory Usage:
Allocated: 3.8 GB
Cached:    4.4 GB


: 

Progress: 100/195,accuracy:2.8,test_loss:2.405690
Test Error: 
 Accuracy: 5.7%, Avg loss: 2.404165 



In [ ]:
#print("Number of layers:", len(activation_monitor.activations))
#for i, activation in enumerate(activation_monitor.activations):
#    print(f"Layer {i + 1}: {activation.shape}")
